# Setup

Install the dependencies:
```sh
pip install gym
pip install gym[atari]
```

# Useful Resources
* [Manual of the game](https://www.gamesdatabase.org/Media/SYSTEM/Atari_2600/Manual/formated/Freeway_-_1981_-_Zellers.pdf)

# Description of the problem

TODO

# Imports

In [1]:
import sys
sys.path.append('../')  # Enable importing from `src` folder

In [2]:
import gym
import time
import src.agents as agents
import src.environment as environment
import src.utils as utils

# Environment

We will be using the Open AI Gym framework in this study.......

In [3]:
env, initial_state = environment.get_env()

print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)

Action Space: Discrete(3)
Observation Space: Box(0, 255, (128,), uint8)


The agent in this game has three possible actions:

* 0: Stay
* 1: Move forward
* 2: Move back

TODO: Talk a bit about the observation space of 128 bytes of RAM...

# Baseline

As a simple baseline, we are using an agent that moves always **up**.

In [4]:
scores = environment.run(agents.Baseline, render=False, n_runs=5)
scores

Score #0: 21
Score #1: 21
Score #2: 23
Score #3: 21
Score #4: 21


[21, 21, 23, 21, 21]

In [5]:
# Mean score
print("Mean score:", sum(scores) / len(scores))

Mean score: 21.4


It usually scores from 21 to 23 points, as shown in the images below:

![Baseline 1](./img/baseline_1.png)
![Baseline 2](./img/baseline_2.png)